In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

In [121]:
df = pd.read_csv('/Users/hongjonghyeok/Downloads/데이터 전처리 과정/최종통합본.csv')

In [122]:
df.columns

Index(['Name', 'Address', 'Latitude', 'Longitude', 'Supply_type',
       'Applicant_type', 'Units', 'Gender', 'Shared', 'Cutline_rate',
       'Cutline_score', 'Year', 'Quarter', 'Applied_type', 'Counts_daiso',
       'Counts_supermarket', 'Counts_laundry', 'Counts_pharmacy',
       'Counts_cafe', 'Counts_convstore', 'Counts_station', 'Infra_score',
       'people', 'Rate1', 'Rate2', 'Rate3'],
      dtype='object')

In [123]:
# 필요한 열 선택 (예: 순위와 가점 예측에 필요한 변수)
features = ['Infra_score', 'Supply_type', 'Applicant_type', 'Units', 'Gender', 'Shared',
            'Year', 'Quarter', 'Applied_type',
            'Counts_daiso', 'Counts_supermarket', 'Counts_laundry', 
            'Counts_pharmacy', 'Counts_cafe', 'Counts_convstore', 'Counts_station', 
            'Rate1', 'Rate2', 'Rate3']
target_rank = 'Cutline_rate'  # 순위 (1, 2, 3)
target_score = 'Cutline_score'  # 가점 (0~10)

In [124]:
# 데이터 분리
X = df[features]
y_rank = df[target_rank]
y_score = df[target_score]

In [125]:
# 범주형 변수 인코딩
X['Supply_type'] = LabelEncoder().fit_transform(X['Supply_type'])
X['Applicant_type'] = LabelEncoder().fit_transform(X['Applicant_type'])
X['Gender'] = LabelEncoder().fit_transform(X['Gender'])
X['Applied_type'] = LabelEncoder().fit_transform(X['Applied_type'])
X['Shared'] = LabelEncoder().fit_transform(X['Shared'])
X['Year'] = LabelEncoder().fit_transform(X['Year'])
X['Quarter'] = LabelEncoder().fit_transform(X['Quarter'])

/var/folders/4g/1y1rn_456xx_f85fqpq2pl0m0000gn/T/ipykernel_39045/586852210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Supply_type'] = LabelEncoder().fit_transform(X['Supply_type'])
/var/folders/4g/1y1rn_456xx_f85fqpq2pl0m0000gn/T/ipykernel_39045/586852210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Applicant_type'] = LabelEncoder().fit_transform(X['Applicant_type'])
/var/folders/4g/1y1rn_456xx_f85fqpq2pl0m0000gn/T/ipykernel_39045/586852210.py:4: SettingWithCopyWarning: 
A value is try

In [126]:
# 학습/테스트 데이터 분리
X_train, X_test, y_rank_train, y_rank_test = train_test_split(X, y_rank, test_size=0.1, random_state=42)
X_train_score, X_test_score, y_score_train, y_score_test = train_test_split(X, y_score, test_size=0.1, random_state=42)

In [127]:
# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [128]:
# 순위 예측 모델 (Random Forest Classifier)
rank_model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=5, verbose=0, random_state=42)
rank_model.fit(X_train, y_rank_train)

# 순위 예측 평가
y_rank_pred = rank_model.predict(X_test)
rank_accuracy = accuracy_score(y_rank_test, y_rank_pred)
print(f"순위 예측 정확도: {rank_accuracy:.2f}")

순위 예측 정확도: 0.82


In [134]:
# 테스트 데이터와 예측 결과를 결합하여 DataFrame 생성
# X_test를 DataFrame으로 변환
X_test_df = pd.DataFrame(X_test, columns=features)  # features는 입력 변수 이름 리스트

# 실제 값과 예측 값을 추가
X_test_df['Actual_Rank'] = y_rank_test.values  # 실제 순위 추가
X_test_df['Predicted_Rank'] = y_rank_pred  # 예측된 순위 추가

# 결과 출력 (상위 5개 행)
print(X_test_df[['Actual_Rank', 'Predicted_Rank']].head())

   Actual_Rank  Predicted_Rank
0            1               2
1            3               1
2            1               1
3            2               2
4            2               1


In [145]:
from sklearn.metrics import mean_squared_error, r2_score

# 순위별로 데이터를 필터링하여 가점 모델 학습
score_models = {}

# y_rank_train과 X_train을 DataFrame으로 변환하고 인덱스 재설정
X_train_df = pd.DataFrame(X_train).reset_index(drop=True)
y_rank_train_df = pd.Series(y_rank_train).reset_index(drop=True)
y_score_train_df = pd.Series(y_score_train).reset_index(drop=True)

for rank in sorted(y_rank_train_df.unique()):
    # 해당 순위의 데이터만 필터링
    rank_indices = (y_rank_train_df == rank)  # Boolean mask
    X_train_rank = X_train_df[rank_indices]
    y_score_train_rank = y_score_train_df[rank_indices]

    # 가점 회귀 모델 (Gradient Boosting Regressor)
    score_model = GradientBoostingRegressor(learning_rate=0.05, max_depth=5, random_state=42)
    score_model.fit(X_train_rank, y_score_train_rank)

    # 모델 저장
    score_models[rank] = score_model

# 테스트 데이터에서 순위별 모델로 가점 예측 수행
for i in range(len(X_test)):
    rank_pred = int(y_rank_pred[i])  # 예측된 순위를 정수로 변환
    score_model = score_models.get(rank_pred)  # 해당 순위의 모델 가져오기
    if score_model:
        # 해당 순위의 모델로 가점 예측
        y_score_pred.append(score_model.predict([X_test[i]])[0])
    else:
        # 해당 순위에 모델이 없을 경우 None 추가
        y_score_pred.append(None)

# None 값을 제외한 유효한 예측값과 실제값 추출
valid_preds = []
valid_actuals = []

for pred, actual in zip(y_score_pred, y_score_test):
    if pred is not None:  # 예측값이 None이 아닌 경우만 추가
        valid_preds.append(pred)
        valid_actuals.append(actual)

# RMSE 계산
rmse = mean_squared_error(valid_actuals, valid_preds, squared=False)

# R² Score 계산
r2 = r2_score(valid_actuals, valid_preds)

# 결과 출력
print(f"\n모델 평가:")
print(f"RMSE (평균 제곱근 오차): {rmse:.2f}")
print(f"R² Score (결정 계수): {r2:.2f}")

# 예측 값과 실제 값 비교 출력
print("\n예측 값과 실제 값 비교 (상위 5개):")
for actual, predicted in zip(valid_actuals[:5], valid_preds[:5]):
    print(f"실제 값: {actual}, 예측 값: {predicted}")


모델 평가:
RMSE (평균 제곱근 오차): 2.17
R² Score (결정 계수): 0.07

예측 값과 실제 값 비교 (상위 5개):
실제 값: 7, 예측 값: 5.468018504967832
실제 값: 4, 예측 값: 1.9590618499185637
실제 값: 6, 예측 값: 5.617959930357705
실제 값: 3, 예측 값: 7.417036439732013
실제 값: 8, 예측 값: 5.487999934700426


/Users/hongjonghyeok/opt/anaconda3/envs/ai_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
